# Projet de Machine Learning : Détection de la Fraude Mobile Money

## 1. Définition du problème

In [3]:

# Objectif : Identifier les transactions suspectes dans les systèmes de paiement mobile (Orange Money, Inwi Money…)
# Jeu de données : Mobile Money Fraud Detection Dataset (IEEE / Zindi Africa Challenge)
# Source : https://zindi.africa/competitions/financial-inclusion-in-africa/data
    

## 2. Importation des librairies et collecte de données

In [4]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Chargement du dataset (après téléchargement depuis Zindi)
data = pd.read_csv('creditcard.csv')
data.head()
    

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## 3. Préparation des données

In [5]:

# Nettoyage et préparation des données
data = data.dropna()
data = data.drop_duplicates()

# Encodage des variables catégorielles
data = pd.get_dummies(data, drop_first=True)

# Séparation des features et de la cible
X = data.drop('Class', axis=1)
y = data['Class']

# Division du jeu de données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    

## 4. Entraînement du modèle de base

In [6]:

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
    

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56651
           1       0.97      0.73      0.83        95

    accuracy                           1.00     56746
   macro avg       0.99      0.86      0.92     56746
weighted avg       1.00      1.00      1.00     56746



## 5. Optimisation automatique des hyperparamètres (Optuna)

In [ ]:

import optuna

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)
    
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return accuracy_score(y_test, preds)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1)

print('Best parameters:', study.best_params)
    

[I 2025-10-28 00:12:30,231] A new study created in memory with name: no-name-7b6e5a31-a7c4-4791-809d-303f71a885c0
[I 2025-10-28 00:16:22,275] Trial 0 finished with value: 0.9994889507630493 and parameters: {'n_estimators': 171, 'max_depth': 7, 'min_samples_split': 7, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.9994889507630493.


Best parameters: {'n_estimators': 171, 'max_depth': 7, 'min_samples_split': 7, 'min_samples_leaf': 3}


In [23]:
import json
#X_test.info()
json_str= X_test.iloc[0].to_json()
print (json_str)

{"Time":61290.0,"V1":1.2288211502,"V2":-0.0634077165,"V3":0.2741451422,"V4":0.6474650218,"V5":-0.0481345612,"V6":0.3720730286,"V7":-0.2242305874,"V8":0.0799390492,"V9":0.6407588171,"V10":-0.2730537022,"V11":-1.2527279388,"V12":0.4650787707,"V13":0.4005021153,"V14":-0.2928418606,"V15":-0.101774016,"V16":-0.3998358978,"V17":0.0343356568,"V18":-0.7835502549,"V19":0.1413449004,"V20":-0.0965659024,"V21":-0.1295544481,"V22":-0.0837793282,"V23":-0.1516614739,"V24":-0.7003715973,"V25":0.5985501645,"V26":0.4914090706,"V27":0.0029892597,"V28":0.0017822861,"Amount":11.5}


## 6. Sauvegarde et déploiement du modèle (Flask + Docker)

In [ ]:

import joblib
from flask import Flask, request, jsonify

# Sauvegarde du modèle
joblib.dump(rf, 'fraud_model.pkl')

# Flask app
app = Flask(__name__)
model = joblib.load('fraud_model.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    df = pd.DataFrame([data])
    prediction = model.predict(df)
    return jsonify({'prediction': int(prediction[0])})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.58.49:5000
Press CTRL+C to quit


## 7. Test de l’API Flask déployée

In [ ]:

import requests

url = 'http://127.0.0.1:5000/predict'
test_data = X_test.iloc[0].to_dict()
response = requests.post(url, json=test_data)
print(response.json())
    

## 8. Dockerfile

In [ ]:

# Contenu du Dockerfile
dockerfile = '''
FROM python:3.10-slim
WORKDIR /app
COPY . /app
RUN pip install -r requirements.txt
EXPOSE 5000
CMD ["python", "app.py"]
'''
with open('Dockerfile', 'w') as f:
    f.write(dockerfile)
    

## 9. requirements.txt

In [ ]:

requirements = '''
pandas
numpy
scikit-learn
flask
joblib
optuna
requests
matplotlib
seaborn
'''
with open('requirements.txt', 'w') as f:
    f.write(requirements)
    